### word2vec을 이용한 단어 embedding

In [ ]:
from gensim.models import Word2Vec
import re

In [ ]:
# GPT야, simpsons 캐릭터 이름이 들어간 랜덤 문장 10개를 생성해줘

sentences = ["Homer Simpson forgot his lunch at home, so he had to buy a burger on his way to work.",
    "Marge was busy knitting a new sweater for Bart's upcoming school play.",
    "Lisa Simpson played a beautiful saxophone solo at the school concert.",
    "Mr. Burns secretly plotted another scheme from his office at the Springfield Nuclear Power Plant.",
    "Ned Flanders offered to help Homer fix the fence between their houses.",
    "Bart Simpson tried a new prank at school, but it didn't go as planned.",
    "Milhouse and Bart spent the afternoon playing video games and forgot to do their homework.",
    "Maggie Simpson's adorable giggle filled the room as she played with her toys.",
    "Apu had a busy day at the Kwik-E-Mart, dealing with a rush of customers.",
    "Krusty the Clown decided to change his show a bit to attract a new audience."]

In [ ]:
# preprocessing
# get rid of stopwords, lower case

sentences = [re.sub(r"[.',]", "", sentence).lower().split(" ") for sentence in sentences]

In [ ]:
sentences[0]

In [ ]:
# train word2vec

skip_gram = Word2Vec(sentences, vector_size=300, min_count=1, window=5, sg=1)

In [ ]:
print("{} 의 vector representation : \n{}".format('homer', skip_gram.wv.get_vector(skip_gram.wv.key_to_index['homer'])))

In [ ]:
skip_gram.wv.most_similar("homer")

직접 유사도 구해보기

In [ ]:
homer_vector = skip_gram.wv.get_vector(skip_gram.wv.key_to_index['homer'])
marge_vector = skip_gram.wv.get_vector(skip_gram.wv.key_to_index['marge'])

In [ ]:
# 유사도 계산하기 from scratch
import numpy as np
from numpy.linalg import norm

def cosine_similarity(vector_a: np.ndarray, vector_b: np.ndarray) -> float:
    """
    두 벡터간 cosine similarity를 계산
    
    Parameters
    ----------
    vector_a : np.ndarray
        The first input vector.
    vector_b : np.ndarray
        The second input vector.

    Returns
    -------
    float
        The cosine similarity between `vector_a` and `vector_b`, which is a value between -1 and 1.

    """

    dot_product = np.dot(vector_a, vector_b)
    norm_a = norm(vector_a)
    norm_b = norm(vector_b)
    similarity = dot_product / (norm_a * norm_b)
    return similarity

In [ ]:
cosine_similarity(homer_vector, marge_vector)

### Simpsons dataset을 활용한 Word2Vec

![](https://images.edrawmax.com/what-is/simpsons-family-tree/example.png) <br>
출처 : https://images.edrawmax.com/what-is/simpsons-family-tree/example.png

In [ ]:
from typing import Optional
import nltk
import re
import pandas as pd
import spacy

nltk.download('punkt')
nltk.download('stopwords')

In [ ]:
df = pd.read_csv('simpsons_dataset.csv')
df.shape

In [ ]:
df.head()

In [ ]:
df.isnull().sum()

In [ ]:
df.loc[0, 'spoken_words']

#### 데이터 전처리

In [ ]:
# lemmatize and remove the stopwords and non-alphabetic characters for each line of dialogue

nlp = spacy.load('en_core_web_sm', disable=['ner', 'parser'])

def cleaning(doc):
    """
    Cleans a spaCy Doc object by lemmatizing its tokens and removing stop words,
    then joins the remaining tokens into a single string if there are more than two tokens left.
    
    Parameters:
    ----------
    doc : spacy.tokens.Doc
        A spaCy Doc object containing the processed text.
    
    Returns:
    ----------
    Optional : str
        A string composed of the lemmatized, non-stop tokens separated by spaces,
        if the resulting list of tokens has more than two elements. Otherwise, returns None.
    """

    txt = [token.lemma_ for token in doc if not token.is_stop]
    if len(txt) > 2:
        return ' '.join(txt)

In [ ]:
# only keep alphabets
cleaner = (re.sub("[^A-Za-z']+", ' ', str(row)).lower() for row in df['spoken_words'])

In [ ]:
txt = [cleaning(doc) for doc in nlp.pipe(cleaner, batch_size=5000)]

In [ ]:
txt[0]

In [ ]:
# dataframe에 넣어서 null이 있는 대화는 삭제
# 주로 null은 특정 행동을 했지만 대화가 없었을 때임

df_clean = pd.DataFrame({'clean': txt})
df_clean = df_clean.dropna().drop_duplicates()
df_clean.shape

In [ ]:
# 하나의 문장을 여러 단위의 단어로 분할
sentences = [s.split(' ') for s in df_clean['clean']]

In [ ]:
len(sentences)

In [ ]:
sentences[0]

#### Word2Vec 모델 훈련

In [ ]:
from gensim.models import Word2Vec

In [ ]:
# help(Word2Vec)

- `window` : 문장 내에서 현재 단어와 예측 단어 사이의 최대 거리. ex) 타겟 단어의 왼쪽과 오른쪽 n번째 단어
- `vector_size` : 단어 벡터의 차원 수
- `min_count` : 이 값보다 총 절대 빈도수가 낮은 모든 단어를 무시함 - (2, 100)
- `sg` : 1은 skip-gram, 0은 CBOW method를 사용

In [ ]:
# 모델 정의 하기
w2v_model = Word2Vec(min_count=20,
                     window=2,
                     vector_size=300,
                     sample=6e-5, 
                     alpha=0.03, 
                     min_alpha=0.0007)

In [ ]:
# 문장에 들어있는 각 단어들을 Word2Vec 모델이 인식할 수 있는 형태로 변환
w2v_model.build_vocab(sentences)

In [ ]:
# 모델 훈련
w2v_model.train(sentences, total_examples=w2v_model.corpus_count, epochs=100)

In [ ]:
help(w2v_model.train)

### 단어간 유사도 확인하기

In [ ]:
# dir(w2v_model.wv)

- most_similar : 주어진 조건에 가장 적합한 단어 탐색
- similarity : 주어진 단어들의 유사도 계산
- doesnt_match : 주어진 단어들 중 가장 '덜 유사한' 단어

In [ ]:
# help(w2v_model.wv.most_similar)

In [ ]:
# help(w2v_model.wv.similarity)

In [ ]:
w2v_model.wv.most_similar(positive=["homer"])

In [ ]:
w2v_model.wv.most_similar(positive=["bart"])

- Woman : homer = ___ : marge

In [ ]:
w2v_model.wv.most_similar(positive=["woman", "homer"], negative=["marge"], topn=3)

In [ ]:
w2v_model.wv.most_similar(positive=["woman", "bart"], negative=["man"], topn=3)

In [ ]:
w2v_model.wv.doesnt_match(['bart', 'homer', 'marge'])

In [ ]:
w2v_model.wv.doesnt_match(['bart', 'lisa', 'marge'])

### 단어 임베딩의 한계점

In [ ]:
bank_vector = w2v_model.wv.get_vector(w2v_model.wv.key_to_index['bank'])

In [ ]:
# bank_vector

In [ ]:
# bank_vector

- 우리가 사용하는 모든 단어는 context에 따라 의미가 다르다
- 단어 embedding의 경우 이런 유연성을 확보하지 못 함
    - 배를 깎아 먹었다 / 배가 고프다 / 배 멀미를 하다

In [ ]:
# bank_vector

### sentence embeddings

In [ ]:
from transformers import BertTokenizer, BertModel
import torch

In [ ]:
# pre-trained model tokenizer와 and bert model
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased') # smaller & uncased model
model = BertModel.from_pretrained('bert-base-uncased')

In [ ]:
# bank가 들어간 유사한 문장 두 개
sentence1 = "I deposited money at the bank."
sentence2 = "The ducks swam to the river bank."

In [ ]:
# 문장을 BERT가 인식할 수 있는 형태로 Tokenize
encoded_input1 = tokenizer(sentence1, return_tensors='pt') # pytorch
encoded_input2 = tokenizer(sentence2, return_tensors='pt')

In [ ]:
encoded_input1

- `input_ids` : 각 단어별로 매핑된 key. 101은 문장의 시작을, 102는 문장의 끝을 의미
- `token_type_ids` : 문장 번호
- `attention_mask` : attention을 가져야 하는 단어는 1, 그렇지 않은 단어는 0. (만약 input이 실제 단어들이라면 1)

In [ ]:
# embedding 생성!
with torch.no_grad():
    output1 = model(**encoded_input1)
    output2 = model(**encoded_input2)

In [ ]:
# embedding 내에서 bank라는 단어 찾아오기 (문장의 5번째에 있는 단어)
bank_embedding_sentence1 = output1.last_hidden_state[0, 5, :]
bank_embedding_sentence2 = output2.last_hidden_state[0, 5, :]

In [ ]:
# cosine similarity 계산

similarity = cosine_similarity(bank_embedding_sentence1, bank_embedding_sentence2)
# print("Embedding for 'bank' in sentence 1:", bank_embedding_sentence1)
# print("Embedding for 'bank' in sentence 2:", bank_embedding_sentence2)
print("Cosine similarity between the two embeddings:", similarity)

--END--